### 使用VAE來做手寫辨識，將最後的結果動態呈現出來，打造最好的呈現方式！

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


### 載入相關模組

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, FloatSlider, widgets, Button
from IPython.display import clear_output

In [3]:
# Keras functions
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, concatenate
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2DTranspose, UpSampling2D
from keras.optimizers import SGD, Adam
from keras import metrics

# Keras dataset
from keras.datasets import mnist

# Keras utilis function
from keras.utils import np_utils

from keras import backend as K

Using TensorFlow backend.


### 載入MNIST資料

In [4]:
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()

print("There are %d training data with size %d x %d" %x_train0.shape)
print("There are %d testing  data with size %d x %d" %x_test0.shape)

There are 60000 training data with size 28 x 28
There are 10000 testing  data with size 28 x 28


In [5]:
# 調整長度
x_train = x_train0.reshape(60000, -1)
x_test = x_test0.reshape(10000, -1)

# 單位化
x_train -= x_train.min()
x_train = x_train/x_train.max()

### VAE

In [6]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(2,), mean=0., stddev=1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [7]:
enc_1 = Dense(100, activation='sigmoid')

enc_mean = Dense(2)
enc_log_var = Dense(2)

dec_2 = Dense(100, activation='sigmoid')
dec_1 = Dense(784, activation='sigmoid')

In [8]:
x = Input(shape=(784,))
enc_x = enc_1(x)

z_mean = enc_mean(enc_x)
z_log_var = enc_log_var(enc_x)

In [9]:
# normal sampling layer
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])

In [10]:
dec_x = dec_2(z)
x_hat = dec_1(dec_x)

In [11]:
VAE = Model(x, x_hat)
VAE.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 100)          78500       input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 2)            202         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 2)            202         dense_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (

### Loss function

In [12]:
def vae_loss(x, x_recon):  
    recovery_loss = 784 * metrics.binary_crossentropy(x, x_recon)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return recovery_loss + kl_loss

### Train VAE

In [13]:
# compile
VAE.compile(loss=vae_loss, optimizer=Adam())

In [14]:
VAE.fit(x_train, x_train, batch_size=32, epochs=10)
# 跑得有點慢改成epochs = 10

Epoch 1/10
60000/60000 [==============================] - 11s 183us/step - loss: 194.8915
Epoch 2/10
60000/60000 [==============================] - 11s 177us/step - loss: 181.2135
Epoch 3/10
60000/60000 [==============================] - 11s 181us/step - loss: 179.5094
Epoch 4/10
60000/60000 [==============================] - 12s 201us/step - loss: 177.0653
Epoch 5/10
60000/60000 [==============================] - 11s 175us/step - loss: 173.6516
Epoch 6/10
60000/60000 [==============================] - 11s 187us/step - loss: 170.5187
Epoch 7/10
60000/60000 [==============================] - 11s 184us/step - loss: 168.1064
Epoch 8/10
60000/60000 [==============================] - 11s 175us/step - loss: 166.2156
Epoch 9/10
60000/60000 [==============================] - 11s 176us/step - loss: 164.3693
Epoch 10/10
60000/60000 [==============================] - 11s 175us/step - loss: 162.8030


In [19]:
# 助教的
VAE.load_weights('VAE_handwriting_model_weights.h5')

### 視覺化呈現

In [20]:
# encoder
VAE_Encoder = Model(x, z_mean)

VAE_Encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 78,702
Trainable params: 78,702
Non-trainable params: 0
_________________________________________________________________


In [21]:
idx = np.random.randint(x_train.shape[0])
print("第 %d 圖的 latent 表示為 %s" %(idx, VAE_Encoder.predict(x_train[idx: idx+1])))

第 34965 圖的 latent 表示為 [[ 2.0493   -1.371318]]


In [22]:
# 分類繪製
VAE_latents = VAE_Encoder.predict(x_test[indices])
plt.scatter(VAE_latents[:, 0], VAE_latents[:, 1], c=y_test0[indices], cmap='tab10')
plt.colorbar()

NameError: name 'indices' is not defined

In [ ]:
# decoder
h_input = Input(shape=(2,))
VAE_Decoder = Model(h_input, dec_1(dec_2(h_input)))
VAE_Decoder.summary()

### decoder視覺化呈現

In [ ]:
def normalized(x):
    x -= x.min()
    x /= x.max()
    return x

In [ ]:
digit_size = 28
n = 15

grid_x_vae = np.linspace(-4+0.05, 4-0.05, n)
grid_y_vae = np.linspace(-4+0.05, 4-0.05, n)
VAE_figure = np.zeros((digit_size * n, digit_size * n))
for i, yi in enumerate(grid_x_vae):
    for j, xi in enumerate(grid_y_vae):
        z_sample = np.array([[xi, yi]])
        x_decoded = VAE_Decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        VAE_figure[(n-i-1) * digit_size: (n - i) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = normalized(digit)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(VAE_figure, cmap='Greys')
plt.axis('off')
plt.show()

### 讓使用者可以挑選兩個數字來進行互動
* 選擇框(輸入框)
* 拉桿

In [ ]:
def pick(x, y):
    number_1 = int(x)
    number_2 = int(y)

    list_1 = np.where(y_train0 == number_1)   
    list_2 = np.where(y_train0 == number_2)

    idx_1 = np.random.choice(list_1[0])    
    idx_2 = np.random.choice(list_2[0]) 
    return idx_1, idx_2

### 使用者選擇數字

In [ ]:
number_1 = widgets.BoundedIntText(
    value=2,
    min=0,
    max=9,
    step=1,
    description='number1:',
    disabled=False
)

number_2 = widgets.BoundedIntText(
    value=7,
    min=0,
    max=9,
    step=1,
    description='number2:',
    disabled=False
)
display(number_1)
display(number_2)    

In [ ]:
idx_1, idx_2 = pick(number_1.value, number_2.value)
print(idx_1)
print(idx_2)

In [ ]:
def inBetween(t):
    
    data_0 = x_train0[idx_1]
    data_1 = x_train0[idx_2]
    data_t = (1-t)*x_train0[idx_1] + t*x_train0[idx_2]

    mu_0 = VAE_Encoder.predict(x_train[idx_1:idx_1+1]).squeeze()
    mu_1 = VAE_Encoder.predict(x_train[idx_2:idx_2+1]).squeeze()
    mu_t = (1-t)*mu_0 + t*mu_1

    plt.figure(figsize=(10, 5))
    ax1 = plt.subplot(2, 1, 2)
    ax1.scatter(mu_0[0], mu_0[1])
    ax1.scatter(mu_1[0], mu_1[1])
    ax1.scatter(mu_t[0], mu_t[1])

    ax2 = plt.subplot(2, 3, 1)
    ax2.imshow(data_0, cmap='Greys')

    ax3 = plt.subplot(2, 3, 2)
    ax3.imshow(data_t, cmap='Greys')

    ax4 = plt.subplot(2, 3, 3)
    ax4.imshow(data_1, cmap='Greys')

In [ ]:
interact(inBetween, t=FloatSlider(value=0.5, 
                                  min=0, 
                                  max=1.0,
                                  step=0.02,))